In [1]:
import re
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import tool
from langchain_teddynote import logging
from dotenv import load_dotenv

load_dotenv()
logging.langsmith("CH-15-Bind-Tools")

@tool
def get_word_length(word: str) -> int:
    """Returns the length of the word"""
    return len(word)

@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together"""
    return a + b

@tool
def naver_new_crawl(news_url: str) -> str:
    """Crawl a 네이버(naver.com) news article and returns the body content."""
    response = requests.get(news_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드 : {response.status_code}")
    return f"{cleaned_title}\n{cleaned_content}"

tools = [get_word_length, add_function, naver_new_crawl]

LangSmith 추적을 시작합니다.
[프로젝트명]
CH-15-Bind-Tools


도구가 바인딩된 LLM에서 invoke() 메서드를 호출한다. 실행 결과는 tool_calls에 저장되는데 name은 도구의 이름을 의미하고 args는 도구에 전달되는 인자를 의미한다.
실행 결과를 보면 답변을 내놓지 않는다. LLM이 어떤 도구를 써야할지만 판단한 상태이다.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

llm_with_tools.invoke("What is the length of 'hello'?").tool_calls

/home/simsim/PycharmProjects/lang_graph_practice/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'name': 'get_word_length',
  'args': {'word': 'hello'},
  'id': 'call_CNq7W2Cav1nCcolrIKvmFY0m',
  'type': 'tool_call'}]

도구 호출 결과를 가지고 직접 도구를 호출해준다.
JsonOutputToolsParser에 도구 목록을 넣어서 체인으로 결합한다.

In [5]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

tool_call_results = chain.invoke("What is the length of 'hello'?")
tool_call_results


[{'args': {'word': 'hello'}, 'type': 'get_word_length'}]

In [9]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :return: 사용 가능한 도구 리스트
    """
    for tool_call_result in tool_call_results:
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]
        # next 함수를 사용하여 일치하는 첫 번째 도구 찾기
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            result = matching_tool.invoke(tool_args)
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")

LCEL 문법으로 연결한다.

In [7]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

chain.invoke("What is the length of 'hello'?")

[실행도구] get_word_length [Argument] {'word': 'hello'}
 [실행결과] 5


In [8]:
chain.invoke("114.5 + 121.2")

[실행도구] add_function [Argument] {'a': 114.5, 'b': 121.2}
 [실행결과] 235.7


In [13]:
chain.invoke("뉴스 기사 내용을 크롤링 해줘: https://n.news.naver.com/mnews/article/001/0015840763?rc=N&ntype=RANKING")

AttributeError: 'NoneType' object has no attribute 'get_text'